In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Fri Mar 25 06:19:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import tensorflow as tf
print(tf.__version__)

2.8.0


In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Computer Vision Projects/The Stanford Dog Bread/train'
valid_path = '/content/drive/MyDrive/Computer Vision Projects/The Stanford Dog Bread/test'

In [ ]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

InceptionV3 = tf.keras.applications.InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# # don't train existing weights
for layer in InceptionV3.layers:
    layer.trainable = False



In [ ]:
  # useful for getting number of output classes
folders = glob('/content/drive/MyDrive/Computer Vision Projects/The Stanford Dog Bread/train/*')

In [ ]:
len(folders)

120

In [ ]:
# our layers - you can add more if you want
x = Flatten()(InceptionV3.output)

In [ ]:
x = Dropout(0.4)(x)
x = Dense(16, activation='relu')(x)
x = BatchNormalization()(x)
# # x = Dropout(0.4)(x)
# # x = Dense(2048, activation='relu')(x)
# # x = BatchNormalization()(x)
x = Dropout(0.2)(x)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=InceptionV3.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_95 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Computer Vision Projects/The Stanford Dog Bread/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 12000 images belonging to 120 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Computer Vision Projects/The Stanford Dog Bread/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 8580 images belonging to 120 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
375/375 [==============================] - 178s 475ms/step - loss: 1.1356 - accuracy: 0.6668 - val_loss: 0.6297 - val_accuracy: 0.8129
Epoch 2/20
375/375 [==============================] - 178s 474ms/step - loss: 1.1266 - accuracy: 0.6679 - val_loss: 0.6131 - val_accuracy: 0.8177
Epoch 3/20
375/375 [==============================] - 177s 472ms/step - loss: 1.0988 - accuracy: 0.6785 - val_loss: 0.6089 - val_accuracy: 0.8212
Epoch 4/20
375/375 [==============================] - 177s 473ms/step - loss: 1.1088 - accuracy: 0.6726 - val_loss: 0.6079 - val_accuracy: 0.8179
Epoch 5/20
375/375 [==============================] - 176s 471ms/step - loss: 1.0999 - accuracy: 0.6705 - val_loss: 0.5974 - val_accuracy: 0.8225
Epoch 6/20
375/375 [==============================] - 177s 472ms/step - loss: 1.1013 - accuracy: 0.6760 - val_loss: 0.6022 - val_accuracy: 0.8207
Epoch 7/20
375/375 [==============================] - 178s 475ms/step - loss: 1.0813 - accuracy: 0.6725 - val_loss: 0.5937 -

In [ ]:
%cd /content/drive/MyDrive/Computer Vision Projects

/content/drive/MyDrive/Computer Vision Projects


In [ ]:
model.save('Inceptionnet12_Epochs.h5')


In [ ]:
# # Convert the model
# converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/Computer Vision Projects/The Stanford Dog Bread/resnet50_saved_model') # path to the SavedModel directory
# tflite_model = converter.convert()

# # Save the model.
# with open('model.tflite', 'wb') as f:
#   f.write(tflite_model)

In [ ]:
%cd /content/drive/MyDrive/Computer Vision Projects/The Stanford Dog Bread

/content/drive/MyDrive/Computer Vision Projects/The Stanford Dog Bread


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('/content/drive/MyDrive/Computer Vision Projects/Inceptionnet_120_Epochs.h5')

In [ ]:
img=image.load_img('/content/drive/MyDrive/Computer Vision Projects/The Stanford Dog Bread/test/n02085620-Chihuahua/n02085620_10074.jpg',target_size=(224,224))

In [ ]:
x=image.img_to_array(img)
x

array([[[ 64.,  48.,  48.],
        [ 68.,  52.,  52.],
        [ 65.,  49.,  50.],
        ...,
        [140., 160.,  97.],
        [142., 164.,  99.],
        [135., 160.,  94.]],

       [[ 60.,  44.,  44.],
        [ 65.,  49.,  49.],
        [ 66.,  50.,  50.],
        ...,
        [116., 133.,  78.],
        [106., 126.,  67.],
        [ 89., 110.,  51.]],

       [[ 64.,  46.,  44.],
        [ 65.,  47.,  45.],
        [ 64.,  46.,  44.],
        ...,
        [120., 133.,  89.],
        [102., 118.,  73.],
        [ 82., 100.,  52.]],

       ...,

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [  6.,   6.,   6.],
        [  8.,   8.,   8.],
        [ 10.,  10.,  10.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 255., 255.],
        ...,
        [  5.,   5.,   5.],
        [  6.,   6.,   6.],
        [  7.,   7.,   7.]],

       [[255., 255., 255.],
        [255., 255., 255.],
        [255., 2

In [ ]:
x.shape

(224, 224, 3)

In [ ]:
x=x/255

In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
model.predict(img_data)

array([[2.25796884e-05, 1.26700569e-03, 4.75402772e-02, 2.29183570e-04,
        2.36916053e-03, 2.30235211e-03, 1.87415237e-04, 7.13917279e-06,
        3.91589850e-03, 4.17375285e-03, 1.44967664e-04, 1.12741254e-04,
        1.67737540e-03, 1.02380138e-04, 1.86946796e-04, 3.48672693e-05,
        2.70700311e-05, 1.39987655e-03, 3.26361018e-03, 1.78937009e-03,
        1.95246737e-03, 2.75799801e-04, 8.82731006e-03, 7.37312832e-04,
        1.44349691e-03, 6.16618665e-04, 4.87374468e-03, 5.06749842e-04,
        1.22057991e-02, 5.57682710e-04, 1.29782525e-03, 5.73312724e-03,
        1.32537950e-02, 2.41406523e-02, 1.62155228e-03, 1.09879707e-03,
        2.02037172e-05, 2.91739847e-03, 2.58845580e-03, 1.14066126e-02,
        5.10007900e-04, 9.02176602e-04, 5.30202351e-06, 3.97638905e-05,
        1.37952133e-03, 3.84039292e-03, 2.46071978e-03, 1.06462529e-02,
        3.31221847e-04, 6.29654969e-04, 8.58161540e-04, 1.51682401e-03,
        8.47213790e-02, 4.84226644e-02, 3.89745296e-03, 9.841343

In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
a

array([69])